# Concurrent and Parallel programming #
```






```
###### Amit Murthy, JuliaCon India 2015 ######
###### https://github.com/amitmurthy ######

## Tasks in Julia ##

- Coroutines / Cooperative multitasking
  - Multi-threading is a WIP under development in the 0.5 branch
- Useful for 
  - IO bound tasks - Network, File, etc
  - Implementation of Timers, background tasks
- High level API - `@async`, `@sync`, `@schedule`

In [ ]:
t0=time()
# Start an async task
t = @async begin
    sleep(2.0)
    return pi
end

# Control comes here immediately
println("Started ", t)

# wait for its completion
resp = wait(t)
println("Response : ", resp, " after ", time()-t0, " seconds")

In [ ]:
# @sync waits for all enclosed async operations to complete
t0=time()
@sync begin
    for i in 1:100
        @async (sleep(1.0); print("$i.."))
    end
end
println("\n\n All tasks finished in ", time() - t0, " seconds")

In [ ]:
# compute-bound or blocking ccalls do not yield
t0=time()
@sync begin
    for i in 1:5
        @async begin
            ccall(:sleep, Cint, (Cint,), 1.0)
            print("$i..")
        end
    end
end
println("\n\n All tasks finished in ", time() - t0, " seconds")

- `@schedule` schedules an expression to be run
  - like `@async` except: 
    - launched task is not waited on by enclosing `@sync` blocks
    - does not "localize" variables

In [1]:
a=1

# @async localizes variables
@async (sleep(2.0); println("@async : ", a))

# @schedule does not
@schedule (sleep(2.0); println("@schedule : ", a))

a="Hello";


@async : 1
@schedule : Hello


##  Tasks - produce/consume ##

- `produce` and `consume` allow a producer task to "feed" one or more consumers
- `produce` blocks till a consumer removes a value
- `consume` blocks till the producer adds a value or exits


In [4]:
function producer(cnt)
    @schedule begin
        for i in 1:cnt
            produce(i)
        end
        println("Producer DONE!")
    end
end

function consumer(t, id)
    @schedule begin
        while !istaskdone(t)
            v = consume(t)
            (v != nothing) && println("consumer $id ===> $v")
        end
        println("consumer $id DONE!")
    end
end

# start a producer task
t=producer(6);

# start consumers
consumer(t, 1);
consumer(t, 2);
consumer(t, 3);

consumer 1 ===> 1
consumer 3 ===> 3
consumer 2 ===> 2
consumer 1 ===> 4
consumer 3 ===> 5
consumer 2 ===> 6
Producer DONE!
consumer 1 DONE!
consumer 3 DONE!
consumer 2 DONE!


In [3]:
# start a producer task
t=producer(6);

for v in t
    println("consumed $v")
end

consumed 1
consumed 2
consumed 3
consumed 4
consumed 5
consumed 6
Producer DONE!


## Channels ##

- Another means of inter-task communication
- Is type-aware and has a size
- API 
  - `put!`
  - `take!`
  - `fetch`
  - `isready`
  - `wait`

In [ ]:
c=Channel{Int64}(100)
for i in 1:6
    put!(c, i)
end

@schedule begin 
    for v in c
        println(v)
    end
    println("Consumer task DONE!")
end;


In [ ]:
close(c);

## Multi-processing ##

- Leverge multiple cores
- Distribute across machines
- Remote function execution as opposed to message passing


## @parallel for ##
- distrubutes the range across workers and executes in parallel 

In [8]:
nprocs() > 1 && rmprocs(workers(); waitfor=0.5)
addprocs(10)
@parallel (+) for i in 1:100
    rand(Bool)
end
@time @parallel (+) for i in 1:10^8
    rand(Bool)
end

49995000

3.989842 seconds (11.48 k allocations: 639.769 KB)


## pmap ##
- distributed `map` function

In [14]:
nprocs() > 1 && rmprocs(workers(); waitfor=0.5)
addprocs(2)
files = filter(x->endswith(x, ".jl"), readdir("./"))

# compilation run
pmap(x->begin
        path=joinpath("/Users/amitm/Julia/julia/base",x)
        readall(`md5 $path`)
    end, files);

# timed run
@time pmap(x->begin
        path=joinpath("/Users/amitm/Julia/julia/base",x)
        # simulate compute time by calculating MD5 a few times
        for i in 1:5
            readall(`md5 $path`)
        end
        readall(`md5 $path`)
    end, files)

1-element Array{Any,1}:
 RemoteException(37,CapturedException(could not spawn `md5 /Users/amitm/Julia/julia/base/JuliaCon_Parallel_Workshop.jl`: no such file or directory (ENOENT),Any[(:_jl_spawn,symbol("process.jl"),262,symbol(""),-1,1),(:anonymous,symbol("process.jl"),415,symbol(""),-1,1),(:setup_stdio,symbol("process.jl"),403,symbol(""),-1,1),(:spawn,symbol("process.jl"),414,symbol(""),-1,1),(:open,symbol("process.jl"),478,symbol(""),-1,1),(:readbytes,symbol("process.jl"),513,symbol(""),-1,1),(:anonymous,symbol("In[14]"),16,symbol(""),-1,1),(:anonymous,symbol("multi.jl"),892,symbol(""),-1,1),(:run_work_thunk,symbol("multi.jl"),645,symbol(""),-1,1),(:anonymous,symbol("multi.jl"),892,symbol("task.jl"),63,1)]))

0.124461 seconds (1.90 k allocations: 95.912 KB)


## distributing work with remote channels ##

In [15]:
nprocs() > 1 && rmprocs(workers(); waitfor=0.5)
addprocs(2)

jobs_c = RemoteRef(()->Channel(100))
results_c = RemoteRef(()->Channel(100))

RemoteRef{Channel{Any}}(1,1,104)

In [17]:
# define function on all workers
@everywhere function do_some_work(jobs_c, results_c)
    println("Worker task started.")
    while true
        try
            rqst = take!(jobs_c)
            if rqst == :EXIT
                break
            else
                sleep(rqst)
                put!(results_c, (myid(), rqst))
            end
        catch e
            println(e)
        end
    end
    println("DONE!")
end

In [18]:
# start the worker tasks....
for p in workers()
    remotecall(p, do_some_work, jobs_c, results_c)
end

	From worker 39:	Worker task started.
	From worker 38:	Worker task started.


In [22]:
# adding jobs....
jobs = rand(1:10, 30)
for i in jobs
    put!(jobs_c, i)
end

# Have the worker tasks exit at the end
for i in 1:nworkers()
    put!(jobs_c, :EXIT)
end


	From worker 39:	DONE!


In [21]:
for i in 1:length(jobs)
    (wrkr, data) = take!(results_c) 
    println("worker $wrkr : ", data)
end

worker 38 : 9
worker 39 : 10
worker 39 : 2
worker 39 : 1
worker 38 : 8
worker 38 : 1
worker 39 : 6
worker 38 : 6
worker 39 : 9
worker 38 : 6
worker 38 : 3
worker 38 : 2
worker 39 : 10
worker 39 : 6
worker 38 : 10
worker 38 : 5
worker 39 : 10
worker 39 : 2
worker 38 : 6
worker 38 : 5
worker 39 : 6
worker 38 : 2
worker 38 : 4
worker 39 : 8
worker 38 : 3
worker 38 : 3
worker 38 : 3
worker 39 : 9
worker 38 : 8
	From worker 38:	DONE!
worker 39 : 6

## More examples ##

In [ ]:
nprocs() > 1 && rmprocs(workers(); waitfor=0.5)
addprocs(4)

# execute expression on all processors
@everywhere println(myid())

In [ ]:
rr = remotecall(workers()[1], ()->(sleep(1.0); pi))
isready(rr)

In [ ]:
fetch(rr)

In [ ]:
take!(rr)

In [ ]:
isready(rr)

## Other calls ##
- `remotecall_fetch` blocks and returns with the result of function execution
- `put!(rr, v)` assigns a value to the reference. Blocks if reference is full
- `@spawnat pid expr` executes expression on worker `pid`
- `@spawn expr` executes expression on the next worker. Cycles through all workers

In [ ]:
nprocs() > 1 && rmprocs(workers(); waitfor=0.5)
addprocs(4)

In [ ]:
# @spawn cycles through all workers
fetch(@spawn myid())

In [ ]:
rr = RemoteRef(()->Channel{AbstractString}(2), workers()[1])
put!(rr, "Hello")

In [ ]:
put!(rr, 2)

In [ ]:
put!(rr, "World")

In [ ]:
# should be ready
isready(rr)

In [ ]:
# take the first element
take!(rr)

In [ ]:
# and then the next
take!(rr)

In [ ]:
# should not be ready
isready(rr)

- RemoteRefs can be serialized across workers

In [ ]:
p1 = workers()[1]

In [ ]:
p2 = workers()[2]

In [ ]:
rr = RemoteRef(p1)

In [ ]:
put!(rr, "put! from $(myid())")

In [ ]:
remotecall_fetch(p2, r->(println(take!(r)); put!(r, "put! from $(myid())"); nothing), rr)

In [ ]:
fetch(rr)